In [1]:
# Supress Warnings

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import the numpy and pandas packages

import numpy as np
import pandas as pd

In [3]:
# TASK 1
#Load the companies and rounds data into two data frames and name them companies and rounds2 respectively.

companies = pd.read_csv("companies.txt", sep="\t", encoding = "ISO-8859-1")
#companies.head()
rounds2 = pd.read_csv("rounds2.csv", sep=",", encoding = "ISO-8859-1")
#rounds2.head()

In [4]:
#Handle encoding and decoding for companies
companies['permalink'] = companies['permalink'].str.encode('utf-8').str.decode('ascii', 'ignore')

In [5]:
#making permalink of companies df into lowercase
companies.permalink=companies.permalink.str.lower()

In [6]:
#deconding the coded for rounds2
rounds2['company_permalink'] = rounds2['company_permalink'].str.encode('utf-8').str.decode('ascii', 'ignore')

In [7]:
#making permalink of rounds2 df into lowercase
rounds2.company_permalink=rounds2.company_permalink.str.lower()

In [8]:
#checking the unique record excepting nulls for rounds2
print(rounds2.company_permalink.nunique())

66368


In [9]:
#checking the unique record excepting nulls for companies
print(companies.permalink.nunique())

66368


In [10]:
#In the companies data frame, which column can be used as the unique key for each company? Write the name of the column.
#Ans:permalink

In [11]:
#Are there any companies in the rounds2 file which are not present in companies? Answer yes or no: Y/N

rounds2.loc[~rounds2['company_permalink'].isin(companies['permalink']),:]

#Since the above statement returns 0 we can conlude that there are no companies in rounds2 that are not in companies

companies_diff = (rounds2.company_permalink).unique is not (companies.permalink).unique
companies_diff

True

In [12]:
#Merging data frame
master_frame = pd.merge(companies, rounds2, how='inner', left_on='permalink',right_on='company_permalink')
master_frame.head(50)
#to get total no of rows
print(len(master_frame.index))

114949


In [13]:
# **** Cleaning the master frame
master_frame=master_frame[(master_frame['raised_amount_usd']).notnull()]

print(round(100*(master_frame.isnull().sum()/len(master_frame.index)),2))
#len(master.index)
#master.raised_amount_usd.isnull().sum()

permalink                   0.00
name                        0.00
homepage_url                4.56
category_list               1.10
status                      0.00
country_code                6.16
state_code                  8.01
region                      7.42
city                        7.42
founded_at                 16.81
company_permalink           0.00
funding_round_permalink     0.00
funding_round_type          0.00
funding_round_code         70.34
funded_at                   0.00
raised_amount_usd           0.00
dtype: float64


In [14]:
# TASK 2
# getting the averages for venture,Angel,Seed,Private_equity
print(round(master_frame[master_frame['funding_round_type'].str.match('venture', case=False)].raised_amount_usd.mean(),2))
print(round(master_frame[master_frame['funding_round_type'].str.match('angel', case=False)].raised_amount_usd.mean(),2))
print(round(master_frame[master_frame['funding_round_type'].str.match('seed', case=False)].raised_amount_usd.mean(),2))
print(round(master_frame[master_frame['funding_round_type'].str.match('private_equity', case=False)].raised_amount_usd.mean(),2))

11748949.13
958694.47
719818.0
73308593.03


In [15]:
#AS Spark Funds invests between 5 Million to 15 Million : Venture Funding is one of the Spark Fund will do.

In [16]:
# TASK 3
#1-Spark Funds wants to see the top nine countries which have received the highest total funding 
#(across ALL sectors for the chosen investment type)
#2- For the chosen investment type, make a data frame named top9 with the top nine countries 
#(based on the total investment amount each country has received)

venture_df=master_frame[master_frame['funding_round_type'].str.match('venture', case=False)];
df_by_country = venture_df.groupby(['country_code'])
top9 = df_by_country['raised_amount_usd'].sum().sort_values(ascending = False).head(9)
top9

country_code
USA    4.225108e+11
CHN    3.983542e+10
GBR    2.024563e+10
IND    1.439186e+10
CAN    9.583332e+09
FRA    7.259537e+09
ISR    6.907515e+09
DEU    6.346960e+09
JPN    3.363677e+09
Name: raised_amount_usd, dtype: float64

In [17]:
# As China is not english Speaking country, The Top3 English Speaking Countries are USA,GBR & IND.

In [18]:
# TASK-4
#1-Extract the primary sector of each category list from the category_list column

#2-Use the mapping file 'mapping.csv' to map each primary sector to one of the eight main sectors 
#(Note that ‘Others’ is also considered one of the main sectors)

mapping = pd.read_csv("mapping.csv",sep=',',  encoding = "ISO-8859-1")

#Removing the null from category list
master_frame=master_frame[(master_frame['category_list']).notnull()]

#Getting the primary category in master data frame
master_frame['primary_sector']=master_frame['category_list'].str.split(pat='|',expand=True)[0]

#cleaning mapping df
mapping=mapping[(mapping['category_list']).notnull()]

#unpivoting the mapping df and getting the main_sector
map_unpivoted=pd.melt(mapping,id_vars=['category_list'],var_name=['main_sector'])
map_unpivoted=map_unpivoted[map_unpivoted.value == 1]

# joining unpivoted mapping file to master dataframe
mapped_category = pd.merge(master_frame,map_unpivoted, left_on='primary_sector',right_on='category_list',how='inner')



In [19]:
# TASK 5
#Create three separate data frames D1, D2 and D3 for each of the three countries containing the observations of funding type FT falling within the 5-15 million USD range. The three data frames should contain:

#All the columns of the master_frame along with the primary sector and the main sector

#The total number (or count) of investments for each main sector in a separate column

#The total amount invested in each main sector in a separate column
D1=mapped_category[(mapped_category['country_code']=='USA') & (mapped_category['funding_round_type'].str.match('venture', case=False))  & 
                (mapped_category.raised_amount_usd >=5000000)  & (mapped_category.raised_amount_usd <=15000000)]
D2=mapped_category[(mapped_category['country_code']=='GBR') & (mapped_category['funding_round_type'].str.match('venture', case=False))  & 
                (mapped_category.raised_amount_usd >=5000000)  & (mapped_category.raised_amount_usd <=15000000)]
D3=mapped_category[(mapped_category['country_code']=='IND') & (mapped_category['funding_round_type'].str.match('venture', case=False))  & 
                (mapped_category.raised_amount_usd >=5000000)  & (mapped_category.raised_amount_usd <=15000000)]

#mapped_category.info()
#master_frame.info()

In [20]:
#The total number (or count) of investments for each main sector
print(len(D1.index))
print(len(D2.index))
print(len(D3.index))

11149
577
299


In [21]:
#The total amount invested
print('Sum of investment in D1 ($M) :' ,D1.raised_amount_usd.sum())
print('Sum of investment in D2 ($M) :' ,D2.raised_amount_usd.sum())
print('Sum of investment in D3 ($M) :' ,D3.raised_amount_usd.sum())

Sum of investment in D1 ($M) : 99661524549.0
Sum of investment in D2 ($M) : 5028704358.0
Sum of investment in D3 ($M) : 2683537552.0


In [22]:
# Top 3 sectors - for USA
by_usa_venture_sector = D1.groupby(['main_sector']).count().sort_values(by = 'raised_amount_usd', ascending = False).head(3)
print(by_usa_venture_sector['company_permalink'])

main_sector
Others                                     2923
Cleantech / Semiconductors                 2297
Social, Finance, Analytics, Advertising    1912
Name: company_permalink, dtype: int64


In [23]:
# Top 3 sectors - for GBR
by_gbr_venture_sector = D2.groupby(['main_sector']).count().sort_values(by = 'raised_amount_usd', ascending = False).head(3)
print(by_gbr_venture_sector['company_permalink'])

main_sector
Others                                     143
Cleantech / Semiconductors                 127
Social, Finance, Analytics, Advertising     98
Name: company_permalink, dtype: int64


In [24]:
# Top 3 sectors - for IND
by_ind_venture_sector = D3.groupby(['main_sector']).count().sort_values(by = 'raised_amount_usd', ascending = False).head(3)
print(by_ind_venture_sector['company_permalink'])

main_sector
Others                        109
News, Search and Messaging     52
Entertainment                  33
Name: company_permalink, dtype: int64


In [89]:
#no of investment in top 3 sectors of USA,GBR,IND
print(by_usa_venture_sector['raised_amount_usd'])
print(by_gbr_venture_sector['raised_amount_usd'])
print(by_ind_venture_sector['raised_amount_usd'])

main_sector
Others                                     2923
Cleantech / Semiconductors                 2297
Social, Finance, Analytics, Advertising    1912
Name: raised_amount_usd, dtype: int64
main_sector
Others                                     143
Cleantech / Semiconductors                 127
Social, Finance, Analytics, Advertising     98
Name: raised_amount_usd, dtype: int64
main_sector
Others                        109
News, Search and Messaging     52
Entertainment                  33
Name: raised_amount_usd, dtype: int64


In [25]:
# Top company in 'Others' sectors - for USA
D1.loc[D1[D1.main_sector == 'Others']['raised_amount_usd'].idxmax()]['permalink']

'/organization/1stdibs'

In [26]:
# Top company in 'Others' sectors - for GBR
D2.loc[D2[D2.main_sector == 'Others']['raised_amount_usd'].idxmax()]['permalink']

'/organization/notonthehighstreet'

In [27]:
# Top company in 'Others' sectors - for IND
D3.loc[D3[D3.main_sector == 'Others']['raised_amount_usd'].idxmax()]['permalink']

'/organization/caratlane'

In [28]:
# Top company in 'Cleantech / Semiconductors' sectors - for USA
D1.loc[D1[D1.main_sector == 'Cleantech / Semiconductors']['raised_amount_usd'].idxmax()]['permalink']

'/organization/480-biomedical'

In [29]:
# Top company in 'Cleantech / Semiconductors' sectors - for GBR
D2.loc[D2[D2.main_sector == 'Cleantech / Semiconductors']['raised_amount_usd'].idxmax()]['permalink']

'/organization/anadigm'

In [30]:
# Top company in 'News, Search and Messaging' sectors - for IND
D3.loc[D3[D3.main_sector == 'News, Search and Messaging']['raised_amount_usd'].idxmax()]['permalink']




'/organization/mobile2win-india'

In [96]:
# Step 6
# Export data frames as csv. Ensure export folder is present
mapped_category.to_csv('./master_frame.csv', sep=',')